<a href="https://colab.research.google.com/github/ThomasTranGithub/Algo.Trading.Strategies/blob/main/RNN_Price_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib as plt
import yfinance as yf
import seaborn as sn

A recurrent neural network (RNN) is a class of artificial neural networks where connections between nodes can create a cycle, allowing output from some nodes to affect subsequent input to the same nodes. This allows it to exhibit temporal dynamic behavior. Derived from feedforward neural networks, RNNs can use their internal state (memory) to process variable length sequences of inputs.[1][2][3] This makes them applicable to tasks such as unsegmented, connected handwriting recognition[4] or speech recognition.[5][6] Recurrent neural networks are theoretically Turing complete and can run arbitrary programs to process arbitrary sequences of inputs.[7]

import the prices for two holdings


we will 

In [76]:
import yfinance as yf

# Define the ticker symbol and time period
ticker_symbol = ['AAPL']
start_date = "2017-01-01"
end_date = "2022-12-31"

# Fetch the stock data
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Print the stock data
print(stock_data)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2017-01-03   28.950001   29.082500   28.690001   29.037500   27.095852   
2017-01-04   28.962500   29.127501   28.937500   29.004999   27.065521   
2017-01-05   28.980000   29.215000   28.952499   29.152500   27.203157   
2017-01-06   29.195000   29.540001   29.117500   29.477501   27.506428   
2017-01-09   29.487499   29.857500   29.485001   29.747499   27.758371   
...                ...         ...         ...         ...         ...   
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.477127   
2022-12-27  131.380005  131.410004  128.720001  130.029999  129.652435   
2022-12-28  129.669998  131.029999  125.870003  126.040001  125.674019   
2022-12-29  127.989998  130.479996  127.730003  129.610001  129.233658   
2022-12-30  128.410004  129.949997  127.430

In [77]:
# Extract adjusted closing prices and volume for AAPL and GOOG
adj_close_prices = stock_data['Adj Close']
volume = stock_data['Volume']

# Print the extracted data
print("Adjusted Closing Prices:")
print(adj_close_prices)

print("\nVolume:")
print(volume)


Adjusted Closing Prices:
Date
2017-01-03     27.095852
2017-01-04     27.065521
2017-01-05     27.203157
2017-01-06     27.506428
2017-01-09     27.758371
                 ...    
2022-12-23    131.477127
2022-12-27    129.652435
2022-12-28    125.674019
2022-12-29    129.233658
2022-12-30    129.552719
Name: Adj Close, Length: 1510, dtype: float64

Volume:
Date
2017-01-03    115127600
2017-01-04     84472400
2017-01-05     88774400
2017-01-06    127007600
2017-01-09    134247600
                ...    
2022-12-23     63814900
2022-12-27     69007800
2022-12-28     85438400
2022-12-29     75703700
2022-12-30     77034200
Name: Volume, Length: 1510, dtype: int64


To use the volume and closing price data as inputs for an RNN (Recurrent Neural Network) model, you can follow these steps:

Import the necessary libraries:

In [78]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

Prepare the input data:

In [79]:
# Assuming you have extracted the volume and adjusted closing prices
volume = stock_data['Volume'].values
closing_price = stock_data['Adj Close'].values

# Normalize the input data
normalized_volume = (volume - np.mean(volume)) / np.std(volume)
normalized_closing_price = (closing_price - np.mean(closing_price)) / np.std(closing_price)

# Combine the normalized volume and closing price into a single input array
input_data = np.column_stack((normalized_volume, normalized_closing_price))


In [80]:
# Assuming you have a target variable indicating the desired prediction
# For example, if you want to predict whether the closing price will increase or decrease the next day:
target = np.where(closing_price[1:] > closing_price[:-1], 1, 0)  # 1: increase, 0: decrease

# Reshape the target array to match the input shape
target = target.reshape(-1, 1)


Prepare the target data:

In [81]:
# Assuming you have defined the training and testing set sizes
train_size = int(0.8 * len(input_data))

x_train = input_data[:train_size]
y_train = target[:train_size]

x_test = input_data[train_size:]
y_test = target[train_size:]


In [82]:
len(x_train)


1208

In [83]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


Build and train the RNN model:

In [84]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))




In [85]:
model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32)



Epoch 1/10
38/38 [==============================] - 3s 5ms/step - loss: 0.6924 - accuracy: 0.5240
Epoch 2/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6899 - accuracy: 0.5430
Epoch 3/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6895 - accuracy: 0.5430
Epoch 4/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6892 - accuracy: 0.5430
Epoch 5/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6893 - accuracy: 0.5406
Epoch 6/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6893 - accuracy: 0.5364
Epoch 7/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6893 - accuracy: 0.5406
Epoch 8/10
38/38 [==============================] - 0s 5ms/step - loss: 0.6894 - accuracy: 0.5414
Epoch 9/10
38/38 [==============================] - 0s 4ms/step - loss: 0.6893 - accuracy: 0.5439
Epoch 10/10
38/38 [==============================] - 0s 4ms/step - loss: 0.6894 - accuracy: 0.5430
